In [1]:
# Importing the libraries
import bs4 as bs
import urllib.request
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import heapq

# Gettings the data source
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/Global_Warming').read()

[nltk_data] Downloading package stopwords to /Users/home/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/home/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# Parsing the data/ creating BeautifulSoup object
soup = bs.BeautifulSoup(source,'lxml')

In [3]:
# Fetching the data
text = ""
for paragraph in soup.find_all('p'):
    text += paragraph.text

In [4]:
# Preprocessing the data
text = re.sub(r'\[[0-9]*\]',' ',text)
text = re.sub(r'\s+',' ',text)
clean_text = text.lower()
clean_text = re.sub(r'\W',' ',clean_text)
clean_text = re.sub(r'\d',' ',clean_text)
clean_text = re.sub(r'\s+',' ',clean_text)

In [5]:
# Tokenize sentences
sentences = nltk.sent_tokenize(text)

In [6]:
# Stopword list
stop_words = nltk.corpus.stopwords.words('english')

In [7]:
# Word counts 
word2count = {}
for word in nltk.word_tokenize(clean_text):
    if word not in stop_words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1

# Converting counts to weights
max_count = max(word2count.values())
for key in word2count.keys():
    word2count[key] = word2count[key]/max_count
    
# Product sentence scores    
sent2score = {}
for sentence in sentences:
    for word in nltk.word_tokenize(sentence.lower()):
        if word in word2count.keys():
            if len(sentence.split(' ')) < 30:
                if sentence not in sent2score.keys():
                    sent2score[sentence] = word2count[word]
                else:
                    sent2score[sentence] += word2count[word]
                    
# Gettings best 10 lines             
best_sentences = heapq.nlargest(10, sent2score, key=sent2score.get)

In [8]:
with open('summary.txt','w') as f:
    for sentence in best_sentences:
        f.write(sentence)
    f.close()
    